In [269]:
import yfinance as yf
import pandas as pd
import numpy as np

ml_df = yf.download('^GSPC', start="1978-01-01", interval="1d")

for i in [1, 2, 3, 5, 7, 14, 21]:
    ml_df[f'Close_{i}_Value'] = ml_df['Adj Close'].pct_change(i)
    ml_df[f'Volume_{i}_Value'] = ml_df['Volume'].pct_change(i)
ml_df.dropna(inplace=True)
for i in [3, 7, 14, 21]:
    ml_df[f'Volt_{i}_Value'] = np.log(1 + ml_df['Close_1_Value']).rolling(i).std()
ml_df.dropna(inplace=True)  

sp500_regimes = pd.read_csv('../data/sp500_regimes.csv')
sp500_regimes.set_index('Date', inplace=True)
sp500_regimes.index = pd.to_datetime(sp500_regimes.index)
ml_df.index = pd.to_datetime(ml_df.index)
data = pd.merge(ml_df.drop(columns=['High','Low','Open','Close','Adj Close']), sp500_regimes, on='Date',how='inner')
data['color'] = data['color'].shift(-1)
data.dropna(inplace=True)

/Users/alden/quantfin/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [270]:
X = data.drop(columns=['color'])
y = data['color']

In [271]:
X.head()
X_columns = X.columns

In [272]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

# Assume `data` is your DataFrame
scaler = StandardScaler()

# Fit on your data and then transform it
X = scaler.fit_transform(X)

# If you want to convert it back to a DataFrame
X = pd.DataFrame(X, columns=X_columns)

# Assume y is your label data
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([0, 0, 0, ..., 0, 0, 0])

In [280]:
# Assume X and y are your data and labels
train_size = int(len(X) * 0.7)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]




In [281]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

# Assuming X is your dataset and y is the target variable
oversampler = SMOTE()  # Choose your oversampler: RandomOverSampler, SMOTE, or ADASYN
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

In [283]:
n_timesteps = 1


# Assuming you have X_resampled_reshaped and y_train as your training data
train_size = int(len(X_resampled) * 0.8)  # 80% for training
X_train, X_val = X_resampled[:train_size], X_resampled[train_size:]
y_train, y_val = y_resampled[:train_size], y_resampled[train_size:]

X_train= np.reshape(X_train,(X_train.shape[0], n_timesteps, X_train.shape[1]//n_timesteps))
X_val = np.reshape(X_val,(X_val.shape[0], n_timesteps, X_val.shape[1]//n_timesteps))
X_test_reshaped = np.reshape(X_test,(X_test.shape[0], n_timesteps, X_test.shape[1]//n_timesteps))

In [219]:
X_train.shape

(15638, 1, 28)

In [322]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2
from keras.layers import GaussianNoise

# Create a Sequential model
model = Sequential()

model.add(LSTM(64, input_shape=(1, X_train.shape[2]), return_sequences=True,kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(GaussianNoise(0.01))
# model.add(Dropout(0.1))
model.add(LSTM(128, return_sequences=True, activation='leaky_relu'))
# model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True, activation='leaky_relu'))

model.add(LSTM(128, return_sequences=True, activation='leaky_relu'))
# model.add(Dropout(0.3))
# model.add(LSTM(16, return_sequences=True, activation='leaky_relu'))
# model.add(LSTM(256, return_sequences=True, activation='leaky_relu'))
# model.add(Dropout(0.4))
# model.add(LSTM(256, return_sequences=True, activation='leaky_relu'))
# model.add(Dropout(0.4))
# model.add(LSTM(16, return_sequences=True, activation='leaky_relu'))
model.add(LSTM(256, activation='leaky_relu',kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001), bias_regularizer=l2(0.001)))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=16,validation_data=(X_val, y_val),callbacks=[early_stopping])

Epoch 1/100
978/978 [==============================] - 13s 10ms/step - loss: 1.4873 - accuracy: 0.5108 - val_loss: 2.3139 - val_accuracy: 0.0000e+00
Epoch 2/100
978/978 [==============================] - 9s 10ms/step - loss: 0.8517 - accuracy: 0.7486 - val_loss: 2.0391 - val_accuracy: 0.0000e+00
Epoch 3/100
978/978 [==============================] - 9s 9ms/step - loss: 0.7427 - accuracy: 0.7718 - val_loss: 2.0537 - val_accuracy: 0.0000e+00
Epoch 4/100
978/978 [==============================] - 9s 10ms/step - loss: 0.6651 - accuracy: 0.7904 - val_loss: 1.8878 - val_accuracy: 0.0000e+00
Epoch 5/100
978/978 [==============================] - 10s 10ms/step - loss: 0.6034 - accuracy: 0.8025 - val_loss: 1.6757 - val_accuracy: 0.0591
Epoch 6/100
978/978 [==============================] - 11s 11ms/step - loss: 0.5517 - accuracy: 0.8158 - val_loss: 1.4137 - val_accuracy: 0.2079
Epoch 7/100
978/978 [==============================] - 10s 10ms/step - loss: 0.5042 - accuracy: 0.8253 - val_loss: 1.3

In [323]:
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=0)

print(f'Test accuracy: {test_accuracy*100}%')

Test accuracy: 92.24515557289124%


In [102]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from bayes_opt import BayesianOptimization
 
# Define a function to build and compile the model
def create_model(lr, neurons):
    model = Sequential()
    model.add(LSTM(int(neurons), input_shape=(1, X_train.shape[2]), return_sequences=True))
    model.add(LSTM(int(neurons)//2, return_sequences=True, activation='leaky_relu'))
    model.add(LSTM(int(neurons)//4, return_sequences=True, activation='leaky_relu'))
    model.add(LSTM(int(neurons)//8, return_sequences=True, activation='leaky_relu'))
    model.add(LSTM(int(neurons)//16, activation='leaky_relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])
    return model

# Define a function for the hyperparameter optimization
def optimize_model(lr, neurons):
    model = create_model(lr, neurons)
    # Assuming you have X_train, y_train as your training data
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    # Assuming you have X_val, y_val as your validation data
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    return val_accuracy

# Define the bounds of the hyperparameters to optimize
bounds = {'lr': (0.0001, 0.01), 'neurons': (128, 1024)}

optimizer = BayesianOptimization(
    f=optimize_model,
    pbounds=bounds,
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=50)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   |    lr     |  neurons  |
-------------------------------------------------


| 1         | 0.9979    | 0.004229  | 773.4     |


| 2         | 0.9753    | 0.0001011 | 398.9     |


| 3         | 0.9979    | 0.001553  | 210.7     |


| 4         | 0.9872    | 0.001944  | 437.6     |


| 5         | 0.9928    | 0.004028  | 610.8     |


KeyboardInterrupt: 

In [327]:
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred, axis=-1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' if you have a multi-class problem
print(f'F1 Score: {f1}')
y_prob_pred = model.predict(X_test_reshaped)
acc_score = accuracy_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob_pred, multi_class='ovr')
print(f'AUC-ROC (One-vs-Rest): {auc_roc}')


100/100 [==============================] - 0s 2ms/step
Accuracy: 92.25%
F1 Score: 0.9274626479008773
100/100 [==============================] - 0s 2ms/step
AUC-ROC (One-vs-Rest): 0.987352306432757
